# AgentCore Memory ブランチングを使用した Strands マルチエージェントシステム

## はじめに

このノートブックでは、AWS AgentCore Memory と Strands フレームワークを使用して、**Memory ブランチングを備えたマルチエージェントシステム**を実装する方法を説明します。この例では、高度な Memory 機能である**会話ブランチング**を紹介します。これにより、エージェントは元の会話スレッドを保持しながら、会話履歴をフォークして代替の会話パスを探索できます。

## チュートリアル詳細

| 項目              | 詳細                                                                             |
|:------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | Memory ブランチングを備えた短期会話                                               |
| エージェントユースケース | 旅行プランニングアシスタント                                                    |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル         | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore 短期 Memory、Strands Agents、ツール経由の Memory 取得            |
| 難易度             | 初級                                                                            |


学習内容：

- 会話履歴をフォークするための Memory ブランチングを実装する方法
- 異なる会話ブランチで動作する特化型エージェントを作成する方法
- ブランチのライフサイクル管理（作成、初期化、ブランチ間の切り替え）
- メインとブランチされた会話の両方で会話コンテキストを維持する方法

### シナリオコンテキスト

この例では、Memory ブランチングを示す**旅行プランニングシステム**を作成します：
1. すべての会話が保存される**メインブランチ**の会話
2. フライトオプションを探索するためにメイン会話からフォークする**フライトエージェントブランチ**
3. ホテルオプションを探索するためにメイン会話からフォークする**ホテルエージェントブランチ**
4. 両方のエージェントはメインブランチから共有された会話履歴にアクセスできます

このアプローチは、Memory ブランチングが以下を可能にすることを示しています：
- **並列探索**：エージェントはメイン会話に影響を与えずに「もしも」シナリオを探索できる
- **コンテキストの保持**：ブランチされた会話は元の会話履歴へのアクセスを維持
- **特化したワークフロー**：各ブランチは元のコンテキストに基づきながら独自の会話パスを辿れる

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件
- Python 3.10 以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory に対する適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境をセットアップし、共有 Memory リソースを作成しましょう！

## ステップ 1: 環境のセットアップ
このノートブックを動作させるために必要なすべてのライブラリをインポートし、クライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

Amazon Bedrock モデルと AgentCore に対する適切な権限を持つリージョンとロールを定義します

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## ステップ 2: 共有 Memory の作成
このセクションでは、特化型エージェント間で共有される Memory リソースを作成します。

In [ ]:
from bedrock_agentcore.memory import MemoryClient

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None


In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Travel Agent STM",         # Human-readable description
        strategies=[],                          # No special memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### ブランチングサポート付き Memory の理解

作成した Memory リソースは**会話ブランチング**をサポートしており、以下を可能にします：

1. **共有セッション、複数ブランチ**：すべてのエージェントは同じ `memory_id`、`actor_id`、`session_id` を使用しますが、異なる `branch_name` 値を持ちます
2. **コンテキストの継承**：ブランチがフォークされると、親ブランチ（通常は「main」）から会話履歴を継承します
3. **独立した進化**：フォーク後、各ブランチは独自の独立した会話スレッドを維持します
4. **ブランチ固有の取得**：エージェントは自分の特定のブランチから Memory を取得できます

このブランチングアプローチにより、特化型エージェントは共有セッション履歴に基づきながら、独自の会話コンテキストを維持できます。

## ステップ 3: ブランチングサポート付き Memory フックプロバイダーの作成

このステップでは、**Memory ブランチング**を実装するカスタム `ShortTermMemoryHook` クラスを定義します。この高度なフックプロバイダーは、基本的な Memory 操作をブランチ管理機能で拡張します：

### 主な機能：
1. **ブランチ管理**：会話ブランチを自動的に作成し初期化
2. **Memory の取得**：指定されたブランチから会話履歴を取得
3. **Memory の保存**：新しい会話を適切なブランチに保存
4. **ブランチのフォーク**：メイン会話スレッドから新しいブランチを作成

### ブランチングの仕組み：
- 各エージェントは `branch_name` を指定できます（デフォルトは「main」）
- main 以外のブランチはメイン会話から自動的にフォークされます
- ブランチはフォークポイントまでの会話履歴を継承します
- 各ブランチはフォーク後、独自の独立した会話フローを維持します

In [ ]:
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory import MemorySessionManager
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_id: str, region_name: str = "us-west-2", branch_name: str = "main"):
        """Initialize the hook with a MemorySessionManager.

        Args:
            memory_id: The AgentCore Memory ID
            region_name: AWS region for the memory service
            branch_name: Branch name for this agent's memory (default: "main")
        """
        self.memory_manager = MemorySessionManager(
            memory_id=memory_id,
            region_name=region_name
        )
        self.memory_id = memory_id
        self.branch_name = branch_name
        self._sessions = {}  # Cache session objects per actor/session combo
        self._branch_initialized = False  # Track if branch has been created

    def _get_or_create_session(self, actor_id: str, session_id: str):
        """Get or create a MemorySession for the given actor/session.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            MemorySession object
        """
        key = f"{actor_id}:{session_id}"
        if key not in self._sessions:
            self._sessions[key] = self.memory_manager.create_memory_session(
                actor_id=actor_id,
                session_id=session_id
            )
        return self._sessions[key]

    def _initialize_branch(self, actor_id: str, session_id: str):
        """Initialize a branch if it doesn't exist and this is not the main branch.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier
        """
        if self._branch_initialized or self.branch_name == "main":
            return

        try:
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Check if branch already exists
            branches = memory_session.list_branches()
            branch_exists = any(b.name == self.branch_name for b in branches)

            if not branch_exists:
                # Get the last event from main branch to fork from
                main_events = memory_session.list_events(branch_name="main")
                if main_events:
                    last_event = main_events[-1]
                    # Create the branch with an initial message
                    memory_session.fork_conversation(
                        root_event_id=last_event.eventId,
                        branch_name=self.branch_name,
                        messages=[
                            ConversationalMessage(f"Starting {self.branch_name} branch", MessageRole.ASSISTANT)
                        ]
                    )
                    logger.info(f"✅ Created branch: {self.branch_name}")

            self._branch_initialized = True

        except Exception as e:
            logger.error(f"Failed to initialize branch {self.branch_name}: {e}", exc_info=True)

    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return

            # Initialize branch if needed (for non-main branches)
            if self.branch_name !="main":
                self._initialize_branch(actor_id, session_id)

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Get last 5 conversation turns from this branch
            recent_turns = memory_session.get_last_k_turns(
                k=5,
                branch_name=self.branch_name
            )

            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message.content.get('role', 'unknown').lower()
                        text = message.content.get('content', {}).get('text', '')
                        if text:
                            context_messages.append(f"{role.title()}: {text}")

                if context_messages:
                    context = "\n".join(context_messages)
                    logger.info(f"Loaded context from branch '{self.branch_name}' ({len(context_messages)} messages)")

                    # Add context to agent's system prompt
                    event.agent.system_prompt += (
                        f"\n\nRecent conversation history (from {self.branch_name}):\n{context}\n\n"
                        "Continue the conversation naturally based on this context."
                    )

                    logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns from branch '{self.branch_name}'")
            else:
                logger.info(f"No previous conversation history found in branch '{self.branch_name}'")

        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}", exc_info=True)

    def on_message_added(self, event: MessageAddedEvent):
        """Store conversation turns in memory on the appropriate branch"""
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return

            # Get the memory session
            memory_session = self._get_or_create_session(actor_id, session_id)

            # Get the last message
            messages = event.agent.messages
            if not messages:
                return

            last_message = messages[-1]
            role_str = last_message.get("role", "").upper()
            content_text = last_message.get("content", [{}])[0].get("text", "")

            if not content_text:
                logger.debug("Skipping empty message")
                return

            # Map role string to MessageRole enum
            role_mapping = {
                "USER": MessageRole.USER,
                "ASSISTANT": MessageRole.ASSISTANT,
                "TOOL": MessageRole.TOOL,
            }
            message_role = role_mapping.get(role_str, MessageRole.USER)

            # Store the message on the appropriate branch
            if self.branch_name == "main":
                # Main branch - just add turns normally
                memory_session.add_turns(
                    messages=[ConversationalMessage(content_text, message_role)]
                )
            else:
                # Non-main branch - need to append to existing branch
                # Initialize branch if it doesn't exist
                if not self._branch_initialized:
                    self._initialize_branch(actor_id, session_id)

                # Get the latest event from this branch
                branch_events = memory_session.list_events(branch_name=self.branch_name)
                if branch_events:
                    # Add to existing branch by specifying branch name (without rootEventId)
                    memory_session.add_turns(
                        messages=[ConversationalMessage(content_text, message_role)],
                        branch={"name": self.branch_name}
                    )
                else:
                    # This shouldn't happen if _initialize_branch worked, but handle it
                    logger.warning(f"Branch {self.branch_name} not found after initialization")
                    self._initialize_branch(actor_id, session_id)

            logger.debug(f"✅ Stored message in branch '{self.branch_name}': {role_str}")

        except Exception as e:
            logger.error(f"Failed to store message: {e}", exc_info=True)

    def create_branch(self, actor_id: str, session_id: str,
                      root_event_id: str, branch_name: str,
                      messages: list):
        """Create a new conversation branch.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier
            root_event_id: Event ID to branch from
            branch_name: Name for the new branch
            messages: List of ConversationalMessage objects to add to the branch
        """
        memory_session = self._get_or_create_session(actor_id, session_id)
        return memory_session.fork_conversation(
            root_event_id=root_event_id,
            branch_name=branch_name,
            messages=messages
        )

    def list_branches(self, actor_id: str, session_id: str):
        """List all branches for a session.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            List of branch information
        """
        memory_session = self._get_or_create_session(actor_id, session_id)
        return memory_session.list_branches()

    def get_session(self, actor_id: str, session_id: str):
        """Get the memory session object for direct access.

        Args:
            actor_id: The actor identifier
            session_id: The session identifier

        Returns:
            MemorySession object
        """
        return self._get_or_create_session(actor_id, session_id)

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks with the registry.

        Args:
            registry: The HookRegistry to register callbacks with
        """
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 4: Memory ブランチングを使用したマルチエージェントアーキテクチャの作成

このセクションでは、ブランチング機能を示すために**異なる Memory ブランチ**を使用する特化型エージェントを作成します：

### ブランチング戦略：
- **メインブランチ**：コーディネーターの会話を保存し、基本会話スレッドとして機能
- **flight_agent_memory ブランチ**：フライト固有の会話用の別ブランチ
- **hotel_agent_memory ブランチ**：ホテル固有の会話用の別ブランチ

各特化型エージェントは独自のブランチで動作し、最初に使用されたときにメイン会話から自動的にフォークされます。これにより以下が可能になります：

- 異なる特化分野に対する独立した会話フロー
- ドメイン固有のコンテキストの分離
- メイン会話スレッドの保持

In [ ]:
# Import the necessary components
from strands import Agent, tool

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
actor_id = f"travel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
namespace = f"travel/{actor_id}/preferences"

### ブランチ Memory を持つ特化型エージェントの作成

次に、システムプロンプトを定義し、異なる Memory ブランチを使用するエージェントを作成します。同じ `actor_id` と `session_id` を使用しながら、異なる `branch_name` 値を使用して分離された会話コンテキストを作成することに注目してください：

In [ ]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner."""

In [ ]:
flight_memory_hooks = None
hotel_memory_hooks = None

### ブランチ固有 Memory を持つエージェントツールの実装

特化型エージェントをツールとして実装します。各エージェントは特定のブランチ名で設定された独自の Memory フックを取得します：
- フライトアシスタントは `flight_agent_memory` ブランチを使用
- ホテルアシスタントは `hotel_agent_memory` ブランチを使用

これらのエージェントが呼び出されると：
1. フックはブランチが存在するか確認
2. 存在しない場合、メイン会話から新しいブランチをフォーク
3. エージェントの会話は専用ブランチに保存
4. エージェントは引き続きメインブランチからコンテキストにアクセス可能

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    global flight_memory_hooks
    try:
        if flight_memory_hooks is None:
            # Create hook with branch name "flight_agent_memory"
            flight_memory_hooks = ShortTermMemoryHook(
                memory_id=memory_id,
                region_name=region,
                branch_name="flight_agent_memory"
            )

        flight_agent = Agent(
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": actor_id, "session_id": session_id}
        )

        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    global hotel_memory_hooks
    try:
        if hotel_memory_hooks is None:
            # Create hook with branch name "hotel_agent_memory"
            hotel_memory_hooks = ShortTermMemoryHook(
                memory_id=memory_id,
                region_name=region,
                branch_name="hotel_agent_memory"
            )

        hotel_booking_agent = Agent(
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": actor_id, "session_id": session_id}
        )

        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### コーディネーターエージェントの作成（メインブランチ）

コーディネーターエージェントは**メインブランチ**で動作し、特化型エージェントに委譲します。以下の点に注目してください：

- フライトまたはホテルアシスタントを呼び出すと、それらのエージェントは独自のブランチをフォークします
- 各特化型エージェントのブランチはメイン会話の現在の状態から開始します

In [ ]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [ ]:
agent_memory_hooks = ShortTermMemoryHook(
                memory_id=memory_id,
                region_name=region,
            )

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    hooks=[agent_memory_hooks],
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant],
    state={
        "actor_id": actor_id,
        "session_id": session_id
    }
)

#### マルチエージェントシステムの準備完了！

## エージェントのテスト

旅行プランニングシナリオでマルチエージェントシステムをテストしましょう：

In [ ]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
response = travel_agent("I would only like to focus on the flight at the moment. Direct flight with British Airways")

In [ ]:
print("\n=== Viewing Memory Branches ===")

if flight_memory_hooks or hotel_memory_hooks:
    # Get any memory session to list branches (they all point to the same session)
    hook = flight_memory_hooks if flight_memory_hooks else hotel_memory_hooks
    if hook:
        memory_session = hook.get_session(actor_id, session_id)

        # List all branches in the session
        branches = memory_session.list_branches()
        print(f"\n📊 Session has {len(branches)} branches total:")
        for branch in branches:
            print(f"  - Branch: {branch.name}")
            print(f"    └─ Events: {len(memory_session.list_events(branch_name=branch.name))}")
            print(f"    └─ Created: {branch.created}")

        print("\n💡 Each branch represents a different agent's memory:")
        print("  • 'main' = Travel coordinator conversations")
        print("  • 'flight_agent_memory' = Flight assistant conversations")
        print("  • 'hotel_agent_memory' = Hotel assistant conversations")

In [ ]:
print("\n=== Accessing Branch-Specific Events ===")

if flight_memory_hooks or hotel_memory_hooks:
    hook = flight_memory_hooks if flight_memory_hooks else hotel_memory_hooks
    if hook:
        memory_session = hook.get_session(actor_id, session_id)

        # Get events from the main branch (coordinator)
        main_events = memory_session.list_events(branch_name="main")
        print(f"\n🌳 Main Branch - Coordinator ({len(main_events)} events):")
        if main_events:
            for event in main_events[-3:]:  # Show last 3 events
                for payload in event.payload:
                    if 'conversational' in payload:
                        role = payload['conversational']['role']
                        text = payload['conversational']['content']['text']
                        print(f"  {role}: {text[:100]}...")
        else:
            print("  No events found in main branch")

        # Get events from the flight agent branch
        try:
            flight_branch_events = memory_session.list_events(branch_name="flight_agent_memory")
            print(f"\n✈️  Flight Agent Branch ({len(flight_branch_events)} events):")
            if flight_branch_events:
                print("All flight-related conversations are stored here:")
                for event in flight_branch_events[-3:]:  # Show last 3 events
                    for payload in event.payload:
                        if 'conversational' in payload:
                            role = payload['conversational']['role']
                            text = payload['conversational']['content']['text']
                            print(f"  {role}: {text[:100]}...")
            else:
                print("  No events found - flight assistant wasn't called yet")
        except Exception as e:
            print(f"  Flight branch not created yet: {e}")

        # Get events from the hotel agent branch
        try:
            hotel_branch_events = memory_session.list_events(branch_name="hotel_agent_memory")
            print(f"\n🏨 Hotel Agent Branch ({len(hotel_branch_events)} events):")
            if hotel_branch_events:
                print("All hotel-related conversations are stored here:")
                for event in hotel_branch_events[-3:]:  # Show last 3 events
                    for payload in event.payload:
                        if 'conversational' in payload:
                            role = payload['conversational']['role']
                            text = payload['conversational']['content']['text']
                            print(f"  {role}: {text[:100]}...")
            else:
                print("  No events found - hotel assistant wasn't called yet")
        except Exception as e:
            print(f"  Hotel branch not created yet: {e}")

## まとめ

このノートブックでは、以下を示しました：

1. **Memory ブランチングの基礎**：AgentCore Memory で会話ブランチを作成・管理する方法
2. **ブランチ固有エージェント**：異なる Memory ブランチで動作する特化型エージェントを実装する方法
3. **自動ブランチ作成**：最初の使用時にブランチがメイン会話から自動的にフォークされる仕組み
4. **ブランチの永続性**：各ブランチが独自の独立した会話履歴を維持する仕組み
5. **コンテキストの継承**：ブランチされた会話がフォークポイントまでのメインブランチからコンテキストを継承する仕組み

### Memory ブランチングの主な利点：
- **分離**：各エージェントは他のエージェントに干渉せずに独自の会話コンテキストを維持
- **柔軟性**：メインスレッドに影響を与えずに代替の会話パスを探索
- **整理**：ドメイン固有の会話を別々のブランチに整理して維持
- **永続性**：ブランチ固有の Memory がエージェントインスタンス間で永続化

この Memory ブランチングアーキテクチャは、異なるエージェントが関連しながらも別々の会話コンテキストを維持する必要がある洗練されたマルチエージェントシステムを構築するための強力なアプローチを提供します。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするために Memory を削除しましょう。

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)